In [2]:
import pandas as pd
from google.cloud import bigquery
import requests
import io
import numpy as np
project_id = 'useful-ward-284119'
client = bigquery.Client(project_id)

c:\Users\lenni\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
# The problem:
# The code attempts to read a CSV file from a remote URL using the `requests` library.
# However, the CSV file has formatting issues in some rows due to quotes and double quotes.
# These problems can cause errors when trying to directly read the CSV file into a DataFrame.

# The proposed solution:
# 1. The `requests` library is used to fetch the content of the CSV file from the provided URL.
# 2. Then, certain characters in the file's content are replaced to correct the formatting issues.
#    - `content.replace('"2','2')`: Instances of '"2' are replaced with '2', removing quotes around the number 2 at the beginning of cells.
#    - `content.replace('";',';')`: Instances of '";' are replaced with ';', removing the double quote followed by a comma at the end of cells.
#    - `content.replace('""','"')`: Instances of '""' are replaced with '"', removing redundant double quotes.
# 3. The corrected CSV file content is read using `pd.read_csv()` from pandas.
#    - `io.StringIO` is used to create an in-memory "file" from the corrected content, which is then passed to `pd.read_csv()`.

# Observations:
# - The proposed solution is one way to address the formatting issue in the CSV file before attempting to read it into a DataFrame.
# - However, this approach may not solve all formatting issues, depending on the complexity of problematic rows in the CSV file.
# - If the CSV file has more complex issues, more advanced techniques may be needed to properly clean and process the data.

#Problem row: "2023-10,975,30-50014329-7,AUTOMOVIL CLUB ARGENTINO,""INTERSECCIÓN RUTAS 3, 33 Y 35"",VILLA BORDEU,BUENOS AIRES,PAMPEANA,19,Gasoil Grade 2,1,Daytime,320.00,23/10/2023,2,YPF,-38.70084,-62.33324";
#Normal row: 2023-10,1267,30-50014329-7,AUTOMOVIL CLUB ARGENTINO,INDEPENDENCIA 3655,MAR DEL PLATA,BUENOS AIRES,PAMPEANA,2,Petrol Unleaded - 95/98 Octane,2,Nighttime,303.00,23/10/2023,2,YPF,-38.01208698,-57.56593606;


csv_file = 'https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/Fuel_Prices_Argentina_2016-2023.csv'
response = requests.get(csv_file)
content = response.text
content = content.replace('"2','2').replace('";',';').replace('""','"')
df = pd.read_csv(io.StringIO(content))
df.head(3)

,Year & Month ID,Company ID (Seller),Legal ID,Company Name,Address,City,Province,Region,Product ID,Product Type,Shift ID,Shift Type,Price (Arg Pesos),Date,Flag Company ID,Flag Company,Latitude,Longitude;
0,2022-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,19,Gasoil Grade 2,1,Daytime,172.5,01/10/2022,28,PUMA,-34.658476,-58.529443;
1,2022-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,19,Gasoil Grade 2,2,Nighttime,172.5,01/10/2022,28,PUMA,-34.658476,-58.529443;
2,2022-10,1376,33-64337382-9,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,PAMPEANA,21,Gasoil Grade 3,1,Daytime,219.0,01/10/2022,28,PUMA,-34.658476,-58.529443;


In [4]:
df[df['Company ID (Seller)']=='']

,Year & Month ID,Company ID (Seller),Legal ID,Company Name,Address,City,Province,Region,Product ID,Product Type,Shift ID,Shift Type,Price (Arg Pesos),Date,Flag Company ID,Flag Company,Latitude,Longitude;


In [5]:
df[df['Company ID (Seller)'].isna()]

,Year & Month ID,Company ID (Seller),Legal ID,Company Name,Address,City,Province,Region,Product ID,Product Type,Shift ID,Shift Type,Price (Arg Pesos),Date,Flag Company ID,Flag Company,Latitude,Longitude;


In [6]:
df.columns = ['yearMonth', 'companyId', 'legalId', 'companyName','adress', 'city', 'province', 'region', 'productId', 'productType','shiftId', 'shiftType', 'price', 'date','flagCompanyId', 'flagCompany', 'latitude', 'longitude']

In [7]:
dtypes = {
'yearMonth':         'string',
'companyId':          'int64',
'legalId':           'string',
'companyName':       'string',
'adress':            'string',
'city':              'string',
'province':          'string',
'region':            'string',
'productId':          'int64',
'productType':       'string',
'shiftId':            'int64',
'shiftType':         'string',
'price':            'float64',
'date':              'datetime64[ns]',
'flagCompanyId':      'int64',
'flagCompany':       'string',
'latitude':         'float64',
'longitude':         'float64'
}
df['longitude'] = df['longitude'].astype('string').str.replace(';','')
df['longitude'] = df['longitude'].replace('',np.nan)
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df = df.astype(dtypes)

In [8]:
df_dim_company =df[['companyId','legalId','companyName','adress','city','province','region']]
df_dim_company = df_dim_company.drop_duplicates()
df_dim_company.groupby('companyId').count().reset_index().sort_values('companyName', ascending=False).head(4)

,companyId,legalId,companyName,adress,city,province,region
0,104,1,1,1,1,1,1
3125,7331,1,1,1,1,1,1
3141,7354,1,1,1,1,1,1
3140,7353,1,1,1,1,1,1


In [9]:
df_dim_product = df[['productId','productType']].drop_duplicates()
df_dim_product

,productId,productType
0,19,Gasoil Grade 2
2,21,Gasoil Grade 3
4,6,Compressed Natural Gas
6,3,Petrol Premium - 95 Octane
8,2,Petrol Unleaded - 95/98 Octane


In [10]:
df_dim_shift=df[['shiftId','shiftType']].drop_duplicates()
df_dim_shift

,shiftId,shiftType
0,1,Daytime
1,2,Nighttime


In [11]:
df_dim_flag_company = df[['flagCompanyId','flagCompany']].drop_duplicates()
df_dim_flag_company

,flagCompanyId,flagCompany
0,28,PUMA
10,4,SHELL C.A.P.S.A.
34,2,YPF
54,26,AXION
70,1,BLANCA
84,29,GULF
278,6,DAPSA S.A.
304,8,REFINOR
552,-1,SIN EMPRESA BANDERA
822,30,VOY


In [12]:
df_ft_fuel = df[['companyId','productId','shiftId','price','date','flagCompanyId','latitude','longitude']]
print(df_ft_fuel.drop_duplicates().shape, df_ft_fuel.shape)

(37084, 8) (37084, 8)


In [31]:
def upload_df_to_bq(df,table_id,url_schema_path):
    content = requests.get(url_schema_path).text
    job_config = bigquery.LoadJobConfig(
        schema=client.schema_from_json(io.StringIO(content)),
        write_disposition="WRITE_TRUNCATE",
    )
    
    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )

    job.result()  
    table = client.get_table(table_id)

In [33]:
upload_df_to_bq(df_dim_shift,'useful-ward-284119.RW_FP20_CH14_ARGENTINE_FUEL_PRICES.DIM_SHIFT','https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/dim_shift_schema.json')
upload_df_to_bq(df_dim_company,'useful-ward-284119.RW_FP20_CH14_ARGENTINE_FUEL_PRICES.DIM_COMPANY','https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/dim_company_schema.json')
upload_df_to_bq(df_dim_flag_company,'useful-ward-284119.RW_FP20_CH14_ARGENTINE_FUEL_PRICES.DIM_FLAG_COMPANY','https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/dim_flag_company_schema.json')
upload_df_to_bq(df_dim_product,'useful-ward-284119.RW_FP20_CH14_ARGENTINE_FUEL_PRICES.DIM_PRODUCT','https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/dim_product_schema.json')
upload_df_to_bq(df_ft_fuel,'useful-ward-284119.RW_FP20_CH14_ARGENTINE_FUEL_PRICES.FT_FUEL','https://raw.githubusercontent.com/ErwinBrath/Power-BI-Projects/main/FP20CH14-National-Fuel-Market-Analysis/ft_fuel_schema.json')